# DCF Modeling

In [1]:
import excelify as el

# Step 0: Assumptions

In [2]:
previous_tax_rates = [6858 / 20564, 4915 / 20116, 4281 / 11460]
assumptions_df = el.ExcelFrame(
    {
        "Company Name": ["Walmart Inc."],
        "Ticker": ["WMT"],
        "Current Share Price": [139.43],
        "Effective Tax Rate": [sum(previous_tax_rates) / len(previous_tax_rates)],
        "Last Fiscal Year": ["2021-01-31"],
    },
)

assumptions_df["Company Name"].set_attributes({"bgcolor": "blue"})
assumptions_df["Current Share Price"].set_attributes({"bgcolor": "#000"})
assumptions_df.transpose(
    include_header=True, header_name="Name", column_names=["Constants"]
)

Name,Constants
Company Name,Walmart Inc.
Ticker,WMT
Current Share Price,139.43
Effective Tax Rate,0.32
Last Fiscal Year,2021-01-31


## Step 1: Unlevered Free Cash Flow

In [3]:
years = ["2019", "2020", "2021"]

In [4]:
def nopat(operating_income, taxes):
    return operating_income + taxes


def deferred_taxes(taxes, percent_book_taxes):
    return -taxes * percent_book_taxes


def unlevered_free_cash_flow(
    nopat, d_and_a, deferred_taxes, other_operating, change_in_working_capial, capex
):
    return (
        nopat
        + d_and_a
        + deferred_taxes
        + other_operating
        + change_in_working_capial
        + capex
    )

def ebitda(ebit, da):
    return ebit + da

def growth_rate(col: el.Col):
    return ((col / col.prev(1)) - 1)

In [5]:
historical_ufcf_df = el.ExcelFrame(
    {
        "Year": years,
        "Retail Square Feet": [1_129, 1_129, 1_121],
        "Net Sales": [510_329.0, 519_926.0, 555_233.0],
        "Membership & Other Income": [4076.0, 4038.0, 3918.0],
        "Operating Income (EBIT)": [21_957.0, 20_568.0, 22_548.0],
        "Depreciation & Amortization": [10_678.0, 10_987.0, 11_152.0],
        "% Book Taxes": [-499 / 4281, 320 / 4915, 1911 / 6858],
        "Other Operating Activities": [1_734, 1_981, 1_521],
        "Change in Working Capital": [295, -327, 7972],
        "Capital Expenditures": [-10_344, -10_705, -10_264],
    }
)

historical_ufcf_df = historical_ufcf_df.with_columns(
    (
        -el.col("Operating Income (EBIT)")
        * el.SingleCellExpr(assumptions_df["Effective Tax Rate"][0])
    ).alias("Taxes, Excluding Effect of Interest"),
    (
        ebitda(el.col("Operating Income (EBIT)"), el.col("Depreciation & Amortization"))
    ).alias("EBITDA"),
)

historical_ufcf_df = historical_ufcf_df.with_columns(
    nopat(
        el.col("Operating Income (EBIT)"),
        el.col("Taxes, Excluding Effect of Interest"),
    ).alias("Net Operating Profit After Tax (NOPAT)"),
    deferred_taxes(
        el.col("Taxes, Excluding Effect of Interest"),
        el.col("% Book Taxes"),
    ).alias("Deferred Taxes"),
)

historical_ufcf_df = historical_ufcf_df.with_columns(
    unlevered_free_cash_flow(
        el.col("Net Operating Profit After Tax (NOPAT)"),
        el.col("Depreciation & Amortization"),
        el.col("Deferred Taxes"),
        el.col("Other Operating Activities"),
        el.col("Change in Working Capital"),
        el.col("Capital Expenditures"),
    ).alias("Annual Unlevered Free Cash Flow"),
    (el.col("Net Sales") + el.col("Membership & Other Income")).alias("Total Revenue"),
)

columns_with_growth_rate = [
    "Retail Square Feet",
    "Sales per Square Foot",
    "COGS and OpEx per Square Foot",
    "Maintenance CapEx per Square Foot",
    "Growth CapEx per New Square Foot",
    "Membership & Other Income",
    "Annual Unlevered Free Cash Flow",
    "EBITDA",
]

historical_ufcf_df = historical_ufcf_df.with_columns(
    (el.col("Net Sales") / el.col("Retail Square Feet")).alias("Sales per Square Foot"),
    (
        (el.col("Net Sales") - el.col("Operating Income (EBIT)"))
        / el.col("Retail Square Feet")
    ).alias("COGS and OpEx per Square Foot"),
    (-el.col("Capital Expenditures") / el.col("Retail Square Feet").prev(1)).alias(
        "Maintenance CapEx per Square Foot"
    ),
    (el.col("Depreciation & Amortization") / el.col("Retail Square Feet")).alias(
        "D&A per Square Foot"
    ),
    (el.ConstantExpr(0.0)).alias("Growth CapEx per New Square Foot"),
    (el.col("Other Operating Activities") / el.col("Total Revenue")).alias(
        "Other Operating Activities (% Revenue)"
    ),
)

# TODO: Think about doing this a bit nicer.
historical_ufcf_df["Maintenance CapEx per Square Foot"][0] = -historical_ufcf_df[
    "Capital Expenditures"
][0] / el.Constant(1158)

# TODO: Find a way to show these in a percentage sigfig.
historical_ufcf_df = historical_ufcf_df.with_columns(
    *[
        growth_rate(el.col(c)).alias(f"{c} Growth Rate")
        for c in columns_with_growth_rate
    ]
)

historical_ufcf_df.evaluate().transpose(
    include_header=True,
    header_name="Name",
    column_names=years,
)

Name,2019,2020,2021
Year,2019.0,2020.0,2021.0
Retail Square Feet,1129.0,1129.0,1121.0
Net Sales,510329.0,519926.0,555233.0
Membership & Other Income,4076.0,4038.0,3918.0
Operating Income (EBIT),21957.0,20568.0,22548.0
Depreciation & Amortization,10678.0,10987.0,11152.0
% Book Taxes,-0.12,0.07,0.28
Other Operating Activities,1734.0,1981.0,1521.0
Change in Working Capital,295.0,-327.0,7972.0
Capital Expenditures,-10344.0,-10705.0,-10264.0


In [6]:
projected_ufcf_df = el.ExcelFrame.empty(columns=historical_ufcf_df.columns, width=10)

input_columns = [
    "Retail Square Feet Growth Rate",
    "Sales per Square Foot Growth Rate",
    "Maintenance CapEx per Square Foot Growth Rate",
    "COGS and OpEx per Square Foot Growth Rate",
    "D&A per Square Foot Growth Rate",
    "Growth CapEx per New Square Foot Growth Rate",
    "Membership & Other Income Growth Rate",
    "% Book Taxes",
    "Other Operating Activities (% Revenue)",
]
inputs = [
    [0.02, 0.02, 0.015, 0.015, 0.01, 0.01, 0.005, 0.005, 0.005, 0.005],
    [0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01],
    [0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01],
    [0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01],
    [0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01, 0.008, 0.008],
    [None, 0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01],
    [0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01],
    [0.075, 0.07, 0.065, 0.06, 0.055, 0.05, 0.05, 0.05, 0.05, 0.05],
    # TODO: Get a more accurate number using sum.
    [0.003 for _ in range(10)],
]
input_attr = {"style": "color: black; background-color: gold;"}


def map(l):
    def fn(idx):
        return el.Constant(l[idx])

    return fn


projected_ufcf_df = projected_ufcf_df.with_columns(
    el.Map(lambda idx: el.Constant(str(2022 + idx))).alias("Year"),
    *[
        el.Map(map(l)).alias(col_name)
        for col_name, l in zip(input_columns, inputs, strict=True)
    ],
)
for col_name in input_columns:
    projected_ufcf_df[col_name].set_attributes(input_attr)

projected_columns = [
    "Growth CapEx per New Square Foot",
    "Retail Square Feet",
    "Sales per Square Foot",
    "COGS and OpEx per Square Foot",
    "Maintenance CapEx per Square Foot",
    "D&A per Square Foot",
    "Membership & Other Income",
]
projected_ufcf_df = projected_ufcf_df.with_columns(
    *[
        (el.col(col).prev(1) * (el.col(f"{col} Growth Rate") + 1.0)).alias(col)
        for col in projected_columns
    ]
)
for col in projected_columns:
    projected_ufcf_df[col][0] = historical_ufcf_df[col][-1].cell_expr * (
        projected_ufcf_df[f"{col} Growth Rate"][0].cell_expr + el.Constant(1.0)
    )


projected_ufcf_df["Growth CapEx per New Square Foot"][0] = 150
projected_ufcf_df["Growth CapEx per New Square Foot"][0].set_attributes(input_attr)

projected_ufcf_df = projected_ufcf_df.with_columns(
    (el.col("Retail Square Feet") * el.col("Sales per Square Foot")).alias("Net Sales"),
    (el.col("Net Sales") + el.col("Membership & Other Income")).alias("Total Revenue"),
    (
        el.col("Net Sales")
        - el.col("COGS and OpEx per Square Foot") * el.col("Retail Square Feet")
    ).alias("Operating Income (EBIT)"),
    (el.col("D&A per Square Foot") * el.col("Retail Square Feet")).alias(
        "Depreciation & Amortization"
    ),
    (el.col("Total Revenue") * el.col("Other Operating Activities (% Revenue)")).alias(
        "Other Operating Activities"
    ),
)

projected_ufcf_df.evaluate().transpose(
    include_header=True,
    header_name="Name",
    column_names=[str(2022 + i) for i in range(10)],
)

Name,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031
Year,2022.0,2023.0,2024.0,2025.0,2026.0,2027.0,2028.0,2029.0,2030.0,2031.0
Retail Square Feet,1143.42,1166.29,1183.78,1201.54,1213.55,1225.69,1231.82,1237.98,1244.17,1250.39
Net Sales,583327.79,612844.18,637587.76,663330.37,683362.94,704000.5,718133.31,732549.84,743574.71,754765.51
Membership & Other Income,4035.54,4156.61,4260.52,4367.03,4454.38,4543.46,4611.61,4680.79,4727.6,4774.87
Operating Income (EBIT),23688.93,24887.59,25892.42,26937.83,27751.35,28589.45,29163.38,29748.83,30196.55,30651.01
Depreciation & Amortization,11659.42,12189.92,12620.22,13065.72,13394.32,13731.19,13937.84,14147.61,14332.09,14518.98
% Book Taxes,0.07,0.07,0.07,0.06,0.06,0.05,0.05,0.05,0.05,0.05
Other Operating Activities,1762.09,1851.0,1925.54,2003.09,2063.45,2125.63,2168.23,2211.69,2244.91,2278.62
Change in Working Capital,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Capital Expenditures,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# TODO

1. Displaying the formula-based ExcelFrame is horribly broken - it assumes that the columns the cells refer to always exist in the table, and it's broken with transpose, too. Fix this.
2. ~~df.select()~~
3. df.transpose() is still a bit ugly - Is there a way to pass column names based on one of my existing columns more easily?
4. `with_columns()` doesn't do any coping on self yet it's a modifying API - think about this.
   1. Ideally, you'd want some sort of a shallow copy.